In [ ]:
import pandas as pd
from nltk.tokenize import word_tokenize, sent_tokenize
import pickle
import numpy as np
from keras.api.preprocessing.sequence import pad_sequences
test = pd.read_csv("work-data/test_nolabel.csv")

In [24]:
def tokenize_text(text):
    return [
        [word.lower() for word in word_tokenize(sentence)] for sentence in sent_tokenize(text)
    ]


In [25]:
tokenized_test = [sent for item in test['text'].apply(tokenize_text) for sent in item]
word_index = pickle.load(open("models/word_index.pkl", "rb"))

In [26]:
def text_to_seq(text, word_index):
    return [word_index[word] for word in word_tokenize(text.lower()) if word in word_index]

max_len = pickle.load(open("models/max_len.pkl", "rb"))
X_test_seq = [text_to_seq(text, word_index) for text in test['text']]
X_test_padded = pad_sequences(X_test_seq, maxlen=max_len, padding='post')

print(X_test_padded)


[[  45   22  134 ...    0    0    0]
 [  26 1126  709 ...    0    0    0]
 [4121    9    4 ...    0    0    0]
 ...
 [ 384 1558  201 ...    0    0    0]
 [   3    3    5 ...    0    0    0]
 [  61    6   10 ...    0    0    0]]


In [27]:
from keras.api.models import load_model

model = load_model("models/lstm-model.keras", compile=False)
y_predict = model.predict(X_test_padded)

56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step


In [28]:
label = np.argmax(y_predict, axis=-1)

label_encoder = pickle.load(open("models/label_encoder.pkl", "rb"))

test['label'] = label_encoder.inverse_transform(label)



In [29]:
test[['id', 'label']].to_csv("work-data/test_label.csv", index=False)